## Segmenting Neighbourhoods in Toronto

#### Installing and Importing Libraries
#### BeautifulSoup4 and lxml for rendering the html data into readable table
#### pandas for manipulating Dataframe

In [1]:
!pip install lxml
!pip install bs4
import lxml as lx
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### Getting website_text from requests library's get function
#### Then converting html to xml content
#### After that, finding the table in the entire page (its a wikitable sortale)
#### Finally, creating empty lists to get the data from the table and converting the lists to a dataframe

In [2]:
#Getting the data
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')
table = soup.find('table',{'class':'wikitable sortable'})
A=[]
B=[]
C=[]
#Creating the dataframe
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True).strip())
        B.append(cells[1].find(text=True).strip())
        C.append(cells[2].find(text=True).strip())
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
#Removing the Not Assigned Boroughs
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df.shape

(103, 3)

In [4]:
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
#Reading the Coordinates file csv
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df_geo.shape

(103, 3)

In [7]:
#Merging the two files
df1 = df.merge(df_geo, left_on='Postal Code', right_on='Postal Code')
df1.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [8]:
print('Number of Unique Boroughs',len(df1['Borough'].unique()))
print('Number of Unique Neighborhoods',len(df1['Neighborhood'].unique()))
print('Shape of Merged DataFrame',df1.shape)

Number of Unique Boroughs 10
Number of Unique Neighborhoods 99
Shape of Merged DataFrame (103, 5)


#### Importing Nominatim Package from geopy.geocoders for getting the coordinates of Toronto

In [9]:
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="Toronto_Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Creating a map of Toronto using Latitude and Longitude Values

In [10]:
# create map of Toronto using latitude and longitude values
import folium # map rendering library
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Defining Client ID, Secret, Version for sending a call to Foursquare

In [15]:
CLIENT_ID = 'MAAYVXJ52QE1JYSBWMI2GNG1LFGY3C1VX2EDYN0JK5OIYPES' # your Foursquare ID
CLIENT_SECRET = 'VKGN1VRZLJKMTK5QLVWY3NFO0YSN0UFP4AH0DDU4YYQNL5UT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('abhaysinghsaini@gmail.com:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100
radius = 600

abhaysinghsaini@gmail.com:
CLIENT_ID: MAAYVXJ52QE1JYSBWMI2GNG1LFGY3C1VX2EDYN0JK5OIYPES
CLIENT_SECRET:VKGN1VRZLJKMTK5QLVWY3NFO0YSN0UFP4AH0DDU4YYQNL5UT


#### Defining a function for getting nearby venues

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Calling the function defined above

In [17]:
toronto_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                 )

toronto_venues

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,PetSmart,43.748639,-79.333488,Pet Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
...,...,...,...,...,...,...,...
2807,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Islington Florist & Nursery,43.630156,-79.518718,Flower Shop
2808,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2809,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Once Upon A Child,43.631075,-79.518290,Kids Store
2810,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


#### Statistical Information on Venue Categories for Neighborhoods

In [18]:
import numpy as np
tva = toronto_venues[['Venue','Neighborhood']].groupby(['Neighborhood']).agg(['count'])
#type(tva)
#tva.columns
#type(tva[('Venue', 'count')])
#tva[('Venue', 'count')].to_numpy()
#print(tva[('Venue','count')])
print('Average Venue Categories in a Neighborhood', tva.mean())
print('Median Venue Categories in a Neighborhood', tva.median())
#tva.mean()
print('Number of Unique Venue Categories',len(toronto_venues['Venue Category'].unique()))
#toronto_venues['Venue Category'].unique()
print('Number of Neighborhoods for which a venue was found',len(toronto_venues['Neighborhood'].unique()))

Average Venue Categories in a Neighborhood Venue  count    28.989691
dtype: float64
Median Venue Categories in a Neighborhood Venue  count    13.0
dtype: float64
Number of Unique Venue Categories 301
Number of Neighborhoods for which a venue was found 97


## Analyze Each Neighborhood

In [19]:
# one hot encoding
# Renaming one venue category as Neighborhood 1 so that it doesn't clash with the Neighborhoods column
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.rename(columns={'Neighborhood': 'Neighborhood1'}, inplace=True)
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
#[toronto_onehot.columns[-1]]
#toronto_onehot.shape
#toronto_onehot.to_csv("venue_info_tor1.csv")

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### getting average value of each venue category for each neighborhood

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
toronto_grouped.shape

(97, 302)

#### Getting top 5 venues for each neighborhood

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store  0.17
1                     Lounge  0.17
2             Breakfast Spot  0.17
3  Latin American Restaurant  0.17
4             Sandwich Place  0.17


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.22
1                 Gym  0.11
2  Athletics & Sports  0.11
3      Sandwich Place  0.11
4         Coffee Shop  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0         Park  0.08
1         Bank  0.08
2  Coffee Shop  0.08
3        Diner  0.04
4  Gas Station  0.04


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.11
1  Italian Restaurant  0.11
2          Restaurant  0.07
3      Sandwich Place  0.07
4  

#### Defining function for returning the most common venues

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Getting the top 10 venues for each neighborhood in a dataframe

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.to_csv('final.csv')
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Clothing Store,Latin American Restaurant,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Pub,Gas Station,Convenience Store,Coffee Shop,Sandwich Place,Gym,Ethiopian Restaurant,Electronics Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Park,Coffee Shop,Deli / Bodega,Diner,Fried Chicken Joint,Sushi Restaurant,Restaurant,Supermarket,Chinese Restaurant
3,Bayview Village,Japanese Restaurant,Café,Chinese Restaurant,Bank,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Restaurant,Liquor Store,Indian Restaurant,Butcher,Pub,Toy / Game Store,Sushi Restaurant


## Cluster Neighborhood

In [25]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
kmeans.labels_

array([0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 0, 1, 1, 2, 0,
       1, 0, 1, 1, 1, 2, 1, 1, 4, 1, 2, 0, 0, 1, 1, 1, 2, 0, 1, 1, 2, 0,
       1, 2, 1, 0, 1, 1, 0, 4, 0, 4, 1, 0, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1,
       4, 1, 1, 1, 0, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 4, 1, 1, 2, 1, 0, 4, 3], dtype=int32)

In [26]:
df1.drop(columns=['Postal Code'], inplace=True)

#### Adding the Cluster Labels to the merged toronto dataframe

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df1

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Construction & Landscaping,Pet Store,Food & Drink Shop,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,North York,Victoria Village,43.725882,-79.315572,2.0,Portuguese Restaurant,Pizza Place,Hockey Arena,Coffee Shop,French Restaurant,Yoga Studio,Drugstore,Distribution Center,Dive Bar,Dog Run
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Theater,Bakery,Park,Café,Restaurant,Breakfast Spot,Ice Cream Shop,Health & Beauty Service
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Vietnamese Restaurant,Accessories Store,Event Space,Miscellaneous Shop,Shoe Store,Korean Restaurant,Furniture / Home Store,Boutique,Cheese Shop
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sandwich Place,Park,Burger Joint,Burrito Place,Falafel Restaurant,Café,Sushi Restaurant,Yoga Studio,Bookstore


## visualize the resulting clusters

#### Discrepancy of 4 neighborhoods in the foursquare dataframe and the original dataframe taken from wikipedia

In [28]:
#neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood'] == 'Islington Avenue, Humber Valley Village']
print('Number of Neighborhoods in the Foursquare Dataframe ',len(neighborhoods_venues_sorted['Neighborhood'].unique()))
print('Number of Neighborhoods in the original, wikipedia dataframe',df1.shape)

Number of Neighborhoods in the Foursquare Dataframe  97
Number of Neighborhoods in the original, wikipedia dataframe (103, 4)


#### Dropping the na values caused due to 4 missing neighborhoods in the foursquare calls for venues

In [29]:
#toronto_merged
#for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
#    print(cluster)
#neighborhoods_venues_sorted['Cluster Labels'].unique()
#toronto_merged.head(20)
#toronto_merged.shape
toronto_merged.dropna(inplace=True)
toronto_merged.head()
#toronto_merged.shape
#toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels']+1
#toronto_merged['Cluster Labels'].dtypes


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Construction & Landscaping,Pet Store,Food & Drink Shop,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,North York,Victoria Village,43.725882,-79.315572,2.0,Portuguese Restaurant,Pizza Place,Hockey Arena,Coffee Shop,French Restaurant,Yoga Studio,Drugstore,Distribution Center,Dive Bar,Dog Run
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Theater,Bakery,Park,Café,Restaurant,Breakfast Spot,Ice Cream Shop,Health & Beauty Service
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Vietnamese Restaurant,Accessories Store,Event Space,Miscellaneous Shop,Shoe Store,Korean Restaurant,Furniture / Home Store,Boutique,Cheese Shop
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sandwich Place,Park,Burger Joint,Burrito Place,Falafel Restaurant,Café,Sushi Restaurant,Yoga Studio,Bookstore


In [30]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    print(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

0.0
2.0
1.0
1.0
1.0
2.0
0.0
1.0
2.0
1.0
2.0
2.0
1.0
1.0
0.0
1.0
1.0
2.0
0.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
3.0
1.0
1.0
1.0
0.0
2.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
1.0
0.0
4.0
0.0
4.0
2.0
4.0
1.0
2.0
1.0
2.0
1.0
1.0
1.0
1.0
2.0
0.0
1.0
1.0
1.0
2.0
4.0
1.0
4.0
1.0
1.0
1.0
2.0
2.0
4.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0


#### Examining Clusters

#### Cluster 1 (Parks, Stations and Shops other than restaurants)

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Construction & Landscaping,Pet Store,Food & Drink Shop,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,"Malvern, Rouge",Fast Food Restaurant,Business Service,Hobby Shop,Spa,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
14,Woodbine Heights,Park,Athletics & Sports,Curling Ice,Bus Stop,Pharmacy,Spa,Beer Store,Skating Rink,Dog Run,Doner Restaurant
18,"Guildwood, Morningside, West Hill",Fast Food Restaurant,Mexican Restaurant,Bank,Medical Center,Intersection,Thrift / Vintage Store,Restaurant,Fried Chicken Joint,Rental Car Location,Park
21,Caledonia-Fairbanks,Park,Sporting Goods Shop,Women's Store,Photography Studio,Gym,Bakery,Mexican Restaurant,Yoga Studio,Dumpling Restaurant,Dive Bar
38,"Kennedy Park, Ionview, East Birchmount Park",Train Station,Discount Store,Convenience Store,Hockey Arena,Bus Station,Coffee Shop,Department Store,Bus Line,Light Rail Station,Yoga Studio
44,"Golden Mile, Clairlea, Oakridge",Intersection,Bakery,Diner,Ice Cream Shop,Metro Station,Convenience Store,Coffee Shop,Bus Station,Yoga Studio,Donut Shop
49,"North Park, Maple Leaf Park, Upwood Park",Bakery,Construction & Landscaping,Trail,Park,Basketball Court,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop
51,"Cliffside, Cliffcrest, Scarborough Village West",Motel,Hardware Store,Chinese Restaurant,American Restaurant,Wings Joint,Farm,Farmers Market,Distribution Center,Dive Bar,Dog Run
56,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Sandwich Place,Convenience Store,Skating Rink,Turkish Restaurant,Discount Store,Event Space,Ethiopian Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 2 (Coffee Shop, followed Restaruants)

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",Coffee Shop,Pub,Theater,Bakery,Park,Café,Restaurant,Breakfast Spot,Ice Cream Shop,Health & Beauty Service
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Vietnamese Restaurant,Accessories Store,Event Space,Miscellaneous Shop,Shoe Store,Korean Restaurant,Furniture / Home Store,Boutique,Cheese Shop
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sandwich Place,Park,Burger Joint,Burrito Place,Falafel Restaurant,Café,Sushi Restaurant,Yoga Studio,Bookstore
7,Don Mills,Japanese Restaurant,Gym,Beer Store,Coffee Shop,Asian Restaurant,Restaurant,Sporting Goods Shop,Discount Store,Dim Sum Restaurant,Chinese Restaurant
9,"Garden District, Ryerson",Coffee Shop,Clothing Store,Italian Restaurant,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Tea Room,Falafel Restaurant,Sandwich Place,Hotel
...,...,...,...,...,...,...,...,...,...,...,...
96,"St. James Town, Cabbagetown",Pizza Place,Coffee Shop,Café,Restaurant,Pet Store,Bakery,Chinese Restaurant,Pub,Park,Japanese Restaurant
97,"First Canadian Place, Underground city",Hotel,Coffee Shop,Café,Restaurant,Japanese Restaurant,Gym,Seafood Restaurant,American Restaurant,Theater,Bakery
98,"The Kingsway, Montgomery Road, Old Mill North",Smoke Shop,Boutique,Bakery,Bar,Coffee Shop,Pub,Liquor Store,River,Dumpling Restaurant,Doner Restaurant
99,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Burger Joint,Café,Yoga Studio,Bubble Tea Shop,Ramen Restaurant


#### Cluster 3 (Pizza Place)

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Portuguese Restaurant,Pizza Place,Hockey Arena,Coffee Shop,French Restaurant,Yoga Studio,Drugstore,Distribution Center,Dive Bar,Dog Run
5,"Islington Avenue, Humber Valley Village",Pharmacy,Shopping Mall,Bank,Grocery Store,Café,Yoga Studio,Donut Shop,Distribution Center,Dive Bar,Dog Run
8,"Parkview Hill, Woodbine Gardens",Pizza Place,Fast Food Restaurant,Bank,Rock Climbing Spot,Gym / Fitness Center,Pharmacy,Café,Breakfast Spot,Athletics & Sports,Gastropub
10,Glencairn,Japanese Restaurant,Pizza Place,Pub,Grocery Store,Dumpling Restaurant,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
11,"West Deane Park, Princess Gardens, Martin Grov...",Pizza Place,Sandwich Place,Tea Room,Chinese Restaurant,Yoga Studio,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
17,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Liquor Store,Pet Store,Electronics Store,Beer Store,Coffee Shop,Café,Pharmacy,Pizza Place,Falafel Restaurant,Farm
32,Scarborough Village,Playground,Middle Eastern Restaurant,Caribbean Restaurant,Basketball Court,Pizza Place,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
50,Humber Summit,Arts & Crafts Store,Pizza Place,Bakery,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
67,Davisville North,Pizza Place,Brewery,Café,Food & Drink Shop,Sushi Restaurant,Hotel,Park,Sandwich Place,Greek Restaurant,Pharmacy
70,Westmount,Pizza Place,Supermarket,Discount Store,Intersection,Chinese Restaurant,Ice Cream Shop,Golf Driving Range,Sandwich Place,Coffee Shop,Flea Market


#### Cluster 4 (Yoga and Martial Arts, Unique)

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,"York Mills, Silver Hills",Martial Arts Dojo,Yoga Studio,Electronics Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


#### Cluster 5 (Parks, Grocery/Convenience Stores)

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Weston,Park,Convenience Store,Grocery Store,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
66,York Mills West,Park,Gym Pool,Convenience Store,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
68,"Forest Hill North & West, Forest Hill Road Park",Jewelry Store,Trail,Sushi Restaurant,Park,Yoga Studio,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
83,"Moore Park, Summerhill East",Park,Gym,Trail,Yoga Studio,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
85,"Milliken, Agincourt North, Steeles East, L'Amo...",Playground,Shopping Plaza,Park,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Diner
91,Rosedale,Park,Playground,Trail,Drugstore,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
